# Experiment 4

In [1]:
### Prob with short periods of time

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
#Read DataFrame
SPY_stocks = pd.read_csv("../data/Preprocessed/SPY_Data.csv")
SPY_stocks = SPY_stocks.drop(['Unnamed: 0'], axis=1)
SPY_stocks.head()

,Date,High,Low,Close
0,1999-07-26,85.789774,84.844432,84.923210
1,1999-07-27,86.469216,85.317081,85.632195
2,1999-07-28,86.538133,85.454929,85.907906
3,1999-07-29,85.238351,84.017284,84.706596
4,1999-07-30,85.297373,83.544552,83.662720


### Generate Input Parameters of NN

In [20]:
inputs_SPY = SPY_stocks.copy()

# --- Simple Moving Averages ---
inputs_SPY['SMA_5'] = inputs_SPY['Close'].rolling(window=5).mean()
inputs_SPY['SMA_20'] = inputs_SPY['Close'].rolling(window=20).mean()
inputs_SPY['SMA_50'] = inputs_SPY['Close'].rolling(window=50).mean()

# --- RSI (14) ---
delta = inputs_SPY['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / (avg_loss + 1e-10)
inputs_SPY['RSI_14'] = 100 - (100 / (1 + rs))

# --- MACD and Signal ---
ema_12 = inputs_SPY['Close'].ewm(span=12, adjust=False).mean()
ema_26 = inputs_SPY['Close'].ewm(span=26, adjust=False).mean()
inputs_SPY['MACD'] = ema_12 - ema_26
inputs_SPY['MACD_Signal'] = inputs_SPY['MACD'].ewm(span=9, adjust=False).mean()

# --- Bollinger Bands (%B) ---
sma_20 = inputs_SPY['Close'].rolling(window=20).mean()
std_20 = inputs_SPY['Close'].rolling(window=20).std()
upper_band = sma_20 + 2 * std_20
lower_band = sma_20 - 2 * std_20
inputs_SPY['Bollinger_b'] = (inputs_SPY['Close'] - lower_band) / (upper_band - lower_band + 1e-10)

# --- ATR (14) ---
high_low = inputs_SPY['High'] - inputs_SPY['Low']
high_close = np.abs(inputs_SPY['High'] - inputs_SPY['Close'].shift())
low_close = np.abs(inputs_SPY['Low'] - inputs_SPY['Close'].shift())
true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
inputs_SPY['ATR_14'] = true_range.rolling(window=14).mean()

# --- Momentum (10) ---
inputs_SPY['Momentum_10'] = inputs_SPY['Close'] - inputs_SPY['Close'].shift(10)

# ---- Parameters of Experiment 3 -----
inputs_SPY['Relative_price'] = inputs_SPY['Close']/inputs_SPY['SMA_20']
inputs_SPY['Norm_Volatily'] = inputs_SPY['ATR_14']/inputs_SPY['Close']
inputs_SPY['Scal_RSI'] = inputs_SPY['RSI_14']/100

inputs_SPY = inputs_SPY.drop(['Low', 'High'], axis=1)

In [21]:
inputs_SPY.head()

,Date,Close,SMA_5,SMA_20,SMA_50,RSI_14,MACD,MACD_Signal,Bollinger_b,ATR_14,Momentum_10,Relative_price,Norm_Volatily,Scal_RSI
0,1999-07-26,84.923210,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,1999-07-27,85.632195,NaN,NaN,NaN,NaN,0.056557,0.011311,NaN,NaN,NaN,NaN,NaN,NaN
2,1999-07-28,85.907906,NaN,NaN,NaN,NaN,0.122218,0.033493,NaN,NaN,NaN,NaN,NaN,NaN
3,1999-07-29,84.706596,NaN,NaN,NaN,NaN,0.076438,0.042082,NaN,NaN,NaN,NaN,NaN,NaN
4,1999-07-30,83.662720,84.966525,NaN,NaN,NaN,-0.043573,0.024951,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
inputs_SPY = inputs_SPY[inputs_SPY['Date'] >= '2000-01-01']
inputs_SPY = inputs_SPY[inputs_SPY['Date'] <= '2025-01-01']
inputs_SPY.head()

,Date,Close,SMA_5,SMA_20,SMA_50,RSI_14,MACD,MACD_Signal,Bollinger_b,ATR_14,Momentum_10,Relative_price,Norm_Volatily,Scal_RSI
112,2000-01-03,92.142548,92.746373,90.878498,88.689529,64.905610,1.209557,1.134688,0.706253,1.204466,1.742256,1.013909,0.013072,0.649056
113,2000-01-04,88.539215,91.930681,90.793552,88.815921,48.025427,0.852350,1.078220,0.150906,1.382858,-1.207680,0.975171,0.015619,0.480254
114,2000-01-05,88.697601,91.067470,90.753062,88.953776,46.416858,0.575409,0.977658,0.190058,1.478556,-2.415451,0.977351,0.016670,0.464169
115,2000-01-06,87.272079,89.940936,90.669935,89.083660,38.638978,0.238158,0.829758,0.025521,1.580544,-4.078476,0.962525,0.018111,0.386390
116,2000-01-07,92.340523,89.798393,90.818503,89.285683,56.171171,0.375536,0.738913,0.711398,1.873088,-0.465248,1.016759,0.020285,0.561712


### Creating the architecture of the Neural Network

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from collections import deque
import math

In [10]:
class DQN(nn.Module):
    def __init__(self, input_dim=17, output_dim=8):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, output_dim)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))  # [batch_size, 256]
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))  # [batch_size, 128]
        x = self.dropout(F.relu(self.fc3(x)))  # [batch_size, 64]
        x = self.fc4(x)  # ¡Esta línea faltaba! [batch_size, 8]
        return x

### Create Custom Trading Environment

In [17]:
#Custom Trading Environment for a single stock
class TradingEnv:
    """
    Initialize the trading environment.
    """
    def __init__(self, df, initial_cash=10_000):
        # Load and sort data
        self.df = df.reset_index(drop=True).copy()
        self.max_steps = len(self.df) - 2  # total steps (days)

        # Initial portfolio settings
        self.initial_cash = initial_cash
        self.cash = initial_cash
        self.shares_held = 0.0
        self.current_step = 0
        self.prev_portfolio_value = initial_cash
        self.returns_history = []

        # Track total reward and history
        self.total_reward = 0
        self.history = []

        # Action mapping
        self.action_mapping = {
            0: 0.05,  # buy small
            1: 0.10,  # buy small - medium
            2: 0.20,  # buy large - medium
            3: 0.35,  # buy large
            4: 0.10,  # sell small
            5: 0.25,  # sell small - medium
            6: 0.50,  # sell large - medium
            7: 1.00   # sell all
        }

        # Define input features
        self.feature_cols = [
            'Close',
            'SMA_5', 'SMA_20', 'SMA_50',
            'RSI_14',
            'MACD', 'MACD_Signal',
            'Bollinger_b',
            'ATR_14',
            'Momentum_10',
            'Relative_price', 'Norm_Volatily', 'Scal_RSI'
        ]

    """
    Reset the environment to the initial state.
    """
    def reset(self):
        self.cash = self.initial_cash          # Reset available capital
        self.shares_held = 0.0                 # No shares at start
        self.current_step = 0                  # Start at the beginning of the dataset
        self.prev_portfolio_value = self.initial_cash  # Track portfolio for reward calc
        self.total_reward = 0.0                # Reset reward tracker
        self.history = []  
        self.returns_history = []# Clear history
    
        return self._get_state()

    """
    Construct the current state vector.
    """
    def _get_state(self):
        row = self.df.loc[self.current_step]
    
        # Extract the market features
        features = []
        for col in self.feature_cols:
            features.append(row[col])
    
        # Append portfolio state
        current_price = self.df.loc[self.current_step, 'Close']
        features.append(self.shares_held * current_price / self.cash)  # ratio position/cash (ADD)
        features.append(self.shares_held * current_price / self.initial_cash)  # % invest portfolio (ADD)
        features.append(self.cash)
        features.append(self.shares_held)
    
        return np.array(features, dtype=np.float32)

    def step(self, action):
        done = False
        reward = 0.0
        invalid_action_penalty = -.01
    
        current_price = self.df.loc[self.current_step, 'Close']
        action_type = self.action_mapping[action]

        #Make Step
        if 0 <= action <= 3:  # buy
            percent = self.action_mapping[action]
            if self.cash >= current_price * action_type:
                self._buy(percent)
        elif 4 <= action <= 7:  # sell
            percent = self.action_mapping[action]
            if self.shares_held > 0:
                self._sell(percent)

        #Get portfolio Value
        portfolio_value = self._get_portfolio_value()
        
        #Calculate Daily Return 
        daily_return = (portfolio_value - self.prev_portfolio_value) / (self.prev_portfolio_value + 1e-6)
        self.returns_history.append(daily_return)

        if len(self.returns_history) > 100:
            self.returns_history.pop(0)

        #Update peak_portfolio
        #self.peak_portfolio = max(self.peak_portfolio, portfolio_value)

        # Calculate reward 
        reward += self.calculate_reward(action)
        self.total_reward += reward

        # Advance to next time step
        self.current_step += 1
        
        #Get new Price of stock
        self.current_price = self.df.loc[self.current_step, 'Close']

        # Update prev_portfolio value
        self.prev_portfolio_value = portfolio_value
    
        # Log history for debugging/analysis
        self.history.append({
            'step': self.current_step,
            'cash': self.cash,
            'shares_held': self.shares_held,
            'portfolio_value': portfolio_value,
            'action': action,
            'reward': reward
        })

        done = self.current_step >= self.max_steps - 1
        if done:

            next_state = self._get_state()
            return next_state, reward, done, {}
    

        next_state = self._get_state()
        return next_state, reward, done, {}


    def calculate_reward(self, action):
        # Inicializar componentes
        rewards = {
            'pnl': 0.0,                  # Profit & Loss básico
            'risk_adjusted': 0.0,        # Retorno ajustado por riesgo
            'invalid_action': 0.0,       # Penalización por acción inválida
            'position_management': 0.0,  # Manejo de posición óptimo
            'trend_alignment': 0.0      # Alineación con tendencia
        }
        
        # 1. Componente PnL (base)
        current_value = self._get_portfolio_value()
        pnl = (current_value - self.prev_portfolio_value) / (self.prev_portfolio_value + 1e-6)
        rewards['pnl'] = np.clip(pnl * 5, -2.0, 2.0)  # Escalado y limitado
        
        # 2. Componente Ajustado por Riesgo (Sharpe-like)
        returns_window = np.array(self.returns_history[-20:])  # Últimos 20 retornos
        if len(returns_window) > 5:
            volatility = np.std(returns_window) + 1e-6
            risk_free = 0.0002  # Tasa libre de riesgo diaria aprox.
            sharpe_like = (np.mean(returns_window) - risk_free) / volatility
            rewards['risk_adjusted'] = np.clip(sharpe_like * 2, -1.5, 1.5)
        
        # 3. Penalización por Acción Inválida (¡Nueva recomendación!)
        invalid_penalty = self._get_invalid_action_penalty(action)
        rewards['invalid_action'] = invalid_penalty
        
        # 4. Gestión de Posición
        rewards['position_management'] = self._calculate_position_score()
        
        # 5. Alineación con Tendencia
        rewards['trend_alignment'] = self._calculate_trend_alignment()
        
        # Ponderación final
        weights = {
            'pnl': 0.4,
            'risk_adjusted': 0.3,
            'invalid_action': 0.15,
            'position_management': 0.1,
            'trend_alignment': 0.05
        }
        
        total_reward = sum(rewards[component] * weights[component] for component in rewards)
        return np.clip(total_reward, -3.0, 3.0)  # Limitar para estabilidad


    def _get_invalid_action_penalty(self, action):
        current_price = self.df.loc[self.current_step, 'Close']
        action_type = self.action_mapping[action]
        
        # Penalización base escalonada
        if (0 <= action <= 3) and (self.cash < current_price * action_type):
            # Intento de compra sin fondos
            base_penalty = -0.5
            
            # Penalización adicional proporcional a lo "ambicioso" de la acción
            ambition_penalty = -0.3 * action_type  # Más fuerte para acciones mayores
            
            return base_penalty + ambition_penalty
        
        elif (4 <= action <= 7) and (self.shares_held <= 0):
            # Intento de venta sin posición
            base_penalty = -0.4
            
            # Penalización por tamaño de orden de venta
            size_penalty = -0.2 * abs(action_type)
            
            return base_penalty + size_penalty
        
        return 0.0  # Acción válida

    def _calculate_position_score(self):
        """Evalúa qué tan óptima es la posición actual"""
        current_price = self.df.loc[self.current_step, 'Close']
        position_ratio = (self.shares_held * current_price) / self._get_portfolio_value()
        
        # Ideal: 30-70% invertido (evita estar totalmente en cash o totalmente invertido)
        optimal_min = 0.3
        optimal_max = 0.7
        
        if position_ratio < optimal_min:
            return -0.5 * (optimal_min - position_ratio)  # Penaliza estar bajo-invertido
        elif position_ratio > optimal_max:
            return -0.8 * (position_ratio - optimal_max)  # Penaliza más el sobre-invertir
        else:
            return 0.3  # Recompensa por estar en rango óptimo

    def _calculate_trend_alignment(self):
        """Recompensa alinear acciones con tendencia del mercado"""
        price_vs_sma = self.df.loc[self.current_step, 'Close'] / self.df.loc[self.current_step, 'SMA_20']
        rsi = self.df.loc[self.current_step, 'RSI_14']
        
        # Lógica de tendencia
        if price_vs_sma > 1.02 and rsi < 70:  # Tendencia alcista saludable
            return 0.4 if self.shares_held > 0 else -0.3
        elif price_vs_sma < 0.98 and rsi > 30:  # Tendencia bajista
            return 0.3 if self.shares_held == 0 else -0.4
        else:
            return 0.1 if abs(self.shares_held) < 0.1 else 0.0  # Mercado lateral

    def _get_portfolio_value(self):
        current_price = self.df.loc[self.current_step, 'Close']
        return self.cash + (self.shares_held * current_price)

    """
    Execute a buy order using a percentage of available cash.
    """
    def _buy(self, percent):
        current_price = self.df.loc[self.current_step, 'Close']
        
        # Capital to use for this transaction
        amount_to_spend = self.cash * percent
    
        # Number of whole shares we can buy
        shares_to_buy = int(amount_to_spend // current_price)
    
        if shares_to_buy > 0:
            self.cash -= shares_to_buy * current_price
            self.shares_held += shares_to_buy

    """
    Execute a sell order using a percentage of held shares.
    """
    def _sell(self, percent):
        current_price = self.df.loc[self.current_step, 'Close']
    
        # Determine how many shares to sell
        shares_to_sell = int(self.shares_held * percent)
    
        if shares_to_sell > 0:
            self.cash += shares_to_sell * current_price
            self.shares_held -= shares_to_sell
    

### Initializing the hyperparameters

In [12]:
# Learning rate for optimizer (controls how fast the model learns)
learning_rate = 3e-4 
# Number of experiences used for each learning step
minibatch_size = 64
# Discount factor (γ): how much future rewards are valued vs. immediate rewards
discount_factor = 0.9
#discount_factor = 0.99
# Size of the experience replay buffer
replay_buffer_size = int(1e6)
# Soft update rate (τ): how fast target network updates towards the main network
interpolation_parameter = 1e-2

### Implementing Experience Replay

In [13]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []

    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self, batch_size):
        experiences = random.sample(self.memory, k=batch_size)
    
        states = torch.from_numpy(
            np.vstack([e[0] for e in experiences if e is not None])
        ).float().to(self.device)
    
        actions = torch.from_numpy(
            np.vstack([e[1] for e in experiences if e is not None])
        ).long().to(self.device)
    
        rewards = torch.from_numpy(
            np.vstack([e[2] for e in experiences if e is not None])
        ).float().to(self.device)
    
        next_states = torch.from_numpy(
            np.vstack([e[3] for e in experiences if e is not None])
        ).float().to(self.device)
    
        dones = torch.from_numpy(
            np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)
        ).float().to(self.device)
    
        return states, next_states, actions, rewards, dones

    def __len__(self):
        return len(self.memory)

### Create DQN Agent

In [14]:
class Agent:
    def __init__(self, state_size, action_size):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        self.local_qnetwork = DQN(state_size, action_size).to(self.device)
        self.target_qnetwork = DQN(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
        self.memory = ReplayMemory(replay_buffer_size)
        self.t_step = 0
        self.best_score = -np.inf
        self.best_model_state = None
        self.validation_interval = 100  # Cada cuántos episodios validar
        self.patience = 5

    def step(self, state, action, reward, next_state, done):
        self.memory.push((state, action, reward, next_state, done))
        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0:
          if len(self.memory.memory) > minibatch_size:
            experiences = self.memory.sample(64)
            self.learn(experiences, discount_factor)

    def act(self, state, epsilon = 0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.local_qnetwork.eval()
        with torch.no_grad():
          action_values = self.local_qnetwork(state)
        self.local_qnetwork.train()
        if random.random() > epsilon:
          return np.argmax(action_values.cpu().data.numpy())
        else:
          return random.choice(np.arange(self.action_size))

    def learn(self, experiences, discount_factor):
        states, next_states, actions, rewards, dones = experiences
        next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
        q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
        q_expected = self.local_qnetwork(states).gather(1, actions)
        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

    def soft_update(self, local_model, target_model, interpolation_parameter):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
          target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

### Initializing iteration of DQN Training

In [35]:
initial_train_year = 2000
test_year = 2005

while test_year <= 2018:
    training_SPY = inputs_SPY.copy()
    training_SPY = training_SPY[training_SPY['Date'] >= str(initial_train_year)+'-01-01']
    training_SPY = training_SPY[training_SPY['Date'] <= str(test_year)+'-01-01']

    testing_SPY = inputs_SPY.copy()
    testing_SPY = testing_SPY[testing_SPY['Date'] >= str(test_year)+'-01-01']
    testing_SPY = testing_SPY[testing_SPY['Date'] <= str(test_year + 1)+'-01-01']

    agent = Agent(17, 8)
    env = TradingEnv(training_SPY)

    print("Training year " + str(test_year))
    number_episodes = 2000
    maximum_number_timesteps_per_episode = 1000
    epsilon_starting_value  = 1.0
    epsilon_ending_value  = 0.01
    #epsilon_ending_value  = 0.1 
    epsilon_decay_value  = 0.995
    #epsilon_decay_value  = 0.95
    epsilon = epsilon_starting_value
    scores_on_400_episodes = deque(maxlen = 400)
    total_value_on_400_episodes = deque(maxlen = 400)
    
    for episode in range(1, number_episodes + 1):
        state = env.reset()
        score = 0
        #for t in range(maximum_number_timesteps_per_episode):
        for t in range(env.max_steps):
            action = agent.act(state, epsilon)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
              break
        scores_on_400_episodes.append(score)
        total_value_on_400_episodes.append(env.prev_portfolio_value)
        epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)

        if episode % 400 == 0:
            print('\rEpisode {}  Average Reward: {:.2f} Average Total Value: {:.2f}'.format(episode, np.mean(scores_on_400_episodes), np.mean(total_value_on_400_episodes)))

    filename = 'Models/dqn_Experiment_4_'+str(test_year)+'.pth'
    torch.save(agent.local_qnetwork.state_dict(), filename)
    test_year += 1


1256
Oldst Date:  2000-01-03
Recent Date:  2004-12-31
252
Oldst Date:  2005-01-03
Recent Date:  2005-12-30
---------------------------------------
1508
Oldst Date:  2000-01-03
Recent Date:  2005-12-30
251
Oldst Date:  2006-01-03
Recent Date:  2006-12-29
---------------------------------------
1759
Oldst Date:  2000-01-03
Recent Date:  2006-12-29
251
Oldst Date:  2007-01-03
Recent Date:  2007-12-31
---------------------------------------
2010
Oldst Date:  2000-01-03
Recent Date:  2007-12-31
253
Oldst Date:  2008-01-02
Recent Date:  2008-12-31
---------------------------------------
2263
Oldst Date:  2000-01-03
Recent Date:  2008-12-31
252
Oldst Date:  2009-01-02
Recent Date:  2009-12-31
---------------------------------------
2515
Oldst Date:  2000-01-03
Recent Date:  2009-12-31
252
Oldst Date:  2010-01-04
Recent Date:  2010-12-31
---------------------------------------
2767
Oldst Date:  2000-01-03
Recent Date:  2010-12-31
252
Oldst Date:  2011-01-03
Recent Date:  2011-12-30
-----------

In [ ]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
#epsilon_ending_value  = 0.1 
epsilon_decay_value  = 0.995
#epsilon_decay_value  = 0.95
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)
total_value_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
    state = env.reset()
    score = 0
    #for t in range(maximum_number_timesteps_per_episode):
    for t in range(env.max_steps):
        action = agent.act(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
          break
    scores_on_100_episodes.append(score)
    total_value_on_100_episodes.append(env.prev_portfolio_value)
    epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
    
    print('\rEpisode {}  Score: {:.2f} Initial Catch: {:.2f}  Total Catch {:.2f}, Total Holds:{}, Price stock: {:.2f}, Total Value: {:.2f}'.format(episode, score, env.initial_cash, env.cash, env.shares_held, env.current_price, env.prev_portfolio_value), end = "")
    
    if episode % 100 == 0:
        print('')
        print('\rEpisode {}  Average Reward: {:.2f} Average Total Value: {:.2f}'.format(episode, np.mean(scores_on_100_episodes), np.mean(total_value_on_100_episodes)))

torch.save(agent.local_qnetwork.state_dict(), 'Models/dqn_trading_Experiment_3.pth')